In [1]:
import numpy as np

import json
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
import keras.backend as K
import tensorflow as tf

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open('data/sentipolc_word_index.json', 'r') as f:
    word_index = json.load(f)

In [3]:
with open('data/words.tsv', 'w') as f:
    for w,_ in word_index.items():
        f.write('{}\n'.format(w))

In [4]:
words = [w for w,_ in word_index.items()]

In [5]:
data = np.load('data/sentipolc_seq.npz')

x_train = data['x_train']
y_train = data['y_train']

x_test = data['x_test']
y_test = data['y_test']

Check the max length of the text

In [6]:
max_len_seq = max([len(x) for x in x_train])
print('max len seq {}'.format(max_len_seq))
max_idx = max(word_index.values())
print('max id {}'.format(max_idx))

max len seq 40
max id 15204


In [7]:
x_train_pad = sequence.pad_sequences(x_train, maxlen=max_len_seq, padding='post')
x_test_pad = sequence.pad_sequences(x_test, maxlen=max_len_seq, padding='post')

In [8]:
x_train_pad.shape

(7410, 40)

In [9]:
x_train_pad[:2]

array([[15204,    11,   980,    15, 15204, 15204,    22, 15204, 15204,
          109,    10, 15204,    34,    33, 15204, 15204, 15204,    47,
        15204,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [15204, 15204,  1381,   474, 15204, 15204, 15204,    47, 15204,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]], dtype=int32)

# Define metrics

In [10]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.
     Computes the precision, a metric for multi-label classification of
     how many selected items are relevant.
      """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))		
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))		
    precision = true_positives / (predicted_positives + K.epsilon())		
    return precision

In [11]:
def recall(y_true, y_pred):
    """Recall metric.
 
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

split_train_val = StratifiedShuffleSplit(1,test_size=0.2, random_state=123456)

for train, test in split_train_val.split(x_train_pad, y_train):
    x_train_pad_split, y_train_pad_split = x_train_pad[train], y_train[train]
    x_val_pad_split, y_val_pad_split = x_train_pad[test], y_train[test]

In [13]:
callbacks = [
    TensorBoard(log_dir='./logs_word_lstm', histogram_freq=1, batch_size=128, embeddings_freq=1, embeddings_metadata=words),
    ReduceLROnPlateau(patience=2, verbose=1)
]

Instructions for updating:
Use the retry module or similar alternatives.


In [14]:
def build_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_idx+1, output_dim=100, input_length=max_len_seq))
    model.add(Bidirectional(GRU(64,  dropout=0.2, recurrent_dropout=0.2, activation='relu')))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    return model

In [15]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 100)           1520500   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               63360     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 1,585,161
Trainable params: 1,585,161
Non-trainable params: 0
_________________________________________________________________


In [16]:
!rm -rf logs_word_lstm
!mkdir -p logs_word_lstm

In [17]:
history = model.fit(x_train_pad_split, y_train_pad_split, validation_data=(x_val_pad_split, y_val_pad_split),
                    batch_size=128, epochs=5, callbacks=callbacks)

Train on 5928 samples, validate on 1482 samples
Epoch 1/5
5928/5928 [==============================] - 8s 1ms/step - loss: 0.6313 - acc: 0.6872 - val_loss: 0.6189 - val_acc: 0.6883
Epoch 2/5
5928/5928 [==============================] - 7s 1ms/step - loss: 0.6116 - acc: 0.6883 - val_loss: 0.6109 - val_acc: 0.6883
Epoch 3/5
5928/5928 [==============================] - 7s 1ms/step - loss: 0.5398 - acc: 0.7056 - val_loss: 0.6051 - val_acc: 0.6970
Epoch 4/5
5928/5928 [==============================] - 7s 1ms/step - loss: 0.4323 - acc: 0.8109 - val_loss: 0.6639 - val_acc: 0.7038
Epoch 5/5
5928/5928 [==============================] - 7s 1ms/step - loss: 0.3359 - acc: 0.8671 - val_loss: 0.7008 - val_acc: 0.6808


In [18]:
del model

## Evaluate on the test set

In [19]:
callbacks = [
    TensorBoard(histogram_freq=0, batch_size=128, embeddings_freq=1, embeddings_metadata=words),
]

In [20]:
model = build_model()

history = model.fit(x_train_pad, y_train, batch_size=128, epochs=3, callbacks=callbacks)

Epoch 1/3
7410/7410 [==============================] - 10s 1ms/step - loss: 0.6364 - acc: 0.6854
Epoch 2/3
7410/7410 [==============================] - 8s 1ms/step - loss: 0.6142 - acc: 0.6876
Epoch 3/3
7410/7410 [==============================] - 7s 975us/step - loss: 0.5316 - acc: 0.7399


In [21]:
loss, acc = model.evaluate(x_test_pad, y_test)

2000/2000 [==============================] - 1s 534us/step


In [22]:
print('loss {}, acc {}'.format(loss, acc))

loss 0.5704997172355651, acc 0.709


In [23]:
#model.save('models/sentipolc_word_lstm.hdf5')

# Overall metrics

In [24]:
from sklearn import metrics

predictions = model.predict_classes(x_train_pad, batch_size=128)
tn, fp, fn, tp = metrics.confusion_matrix(y_train, predictions).ravel()

recall = tp/(tp+fp)
precision = tp/(tp+fn)
f1 = metrics.f1_score(y_train,predictions)

print ('precision: {}'.format(precision), '\nrecall: {}'.format(recall), '\nf1: {}'.format(f1))

precision: 0.5774891774891775 
recall: 0.8109422492401216 
f1: 0.6745891276864727
